<a href="https://colab.research.google.com/github/sp7412/colab/blob/master/distilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [120]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def softmax_sparse_categorical_crossentropy(labels, logits):
  softmaxed = tf.keras.backend.softmax(logits)

  return tf.keras.losses.sparse_categorical_crossentropy(labels, softmaxed)

In [121]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [122]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [123]:
ds_info.features['image'].shape

(28, 28, 1)

In [124]:
num_training_examples = ds_info.splits['train'].num_examples

In [125]:
ds_train_iter = ds_train.as_numpy_iterator()
train_images, train_labels = zip(*[ds_train_iter.next() for i in range(num_training_examples)])
# train_images, train_labels = zip(*[ds_train_iter.next() for i in range(10)])

In [126]:
train_images = np.asarray(train_images)

In [127]:
train_images.shape

(60000, 28, 28, 1)

In [128]:
# ds_train = ds_train.take(10)

In [129]:
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [130]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [131]:
control_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#control_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
control_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
control_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_11 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_11 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_12 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_12 (Dropout)         (None, 6, 6, 64)         

In [70]:
control_model.fit(ds_train, epochs=5)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.9824 - accuracy: 0.7300
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.3814 - accuracy: 0.9057
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2623 - accuracy: 0.9297
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2174 - accuracy: 0.9387
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.1917 - accuracy: 0.9440


In [71]:
test_loss, test_acc = control_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

79/79 [==============================] - 2s 19ms/step - loss: 0.1754 - accuracy: 0.9486
Test Loss: 0.17537686228752136
Test Accuracy: 0.9485999941825867


In [26]:
teacher_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#teacher_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
teacher_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
teacher_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_3 (Batch (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 6, 64)         

In [27]:
teacher_model.fit(ds_train, epochs=5)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.3957 - accuracy: 0.8915
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.1242 - accuracy: 0.9649
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0895 - accuracy: 0.9741
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0727 - accuracy: 0.9775
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0622 - accuracy: 0.9804


In [28]:
test_loss, test_acc = teacher_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

79/79 [==============================] - 2s 20ms/step - loss: 0.0383 - accuracy: 0.9886
Test Loss: 0.03831847384572029
Test Accuracy: 0.9886000156402588


In [111]:
train_images = list(map(lambda x:tf.reshape(x,(-1,28,28,1)), train_images))

In [132]:
ds_train_images = tf.data.Dataset.from_tensor_slices(list(map(lambda x:tf.reshape(x,(-1,28,28,1)), train_images)))

In [133]:
ds_train_images

<TensorSliceDataset shapes: (1, 28, 28, 1), types: tf.float32>

In [134]:
soft_labels = teacher_model.predict(ds_train_images,verbose=1)

60000/60000 [==============================] - 82s 1ms/step


In [135]:
temperature = 3
afterwards_temperature = 1

def temperature_softmax(logits):
  soft_logits = tf.keras.backend.exp(logits / temperature)
  return soft_logits / tf.keras.backend.sum(soft_logits, axis=-1, keepdims=True) / afterwards_temperature

def distillation_loss(labels, logits):
  labels = temperature_softmax(labels)
  logits = temperature_softmax(logits)

  return -tf.keras.backend.mean(labels * tf.keras.backend.log(logits))

In [136]:
student_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  # tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

student_model.compile('adam', distillation_loss, ['accuracy'])
student_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_13 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_13 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_14 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_14 (Dropout)         (None, 6, 6, 64)         

In [96]:
tf.shape(train_images)[0],tf.shape(soft_labels)[0]

(<tf.Tensor: shape=(), dtype=int32, numpy=60000>,
 <tf.Tensor: shape=(), dtype=int32, numpy=60000>)

In [115]:
np.asarray(train_images).shape

(10, 1, 28, 28, 1)

In [116]:
soft_labels.shape

(10, 10)

In [ ]:
student_model.fit(train_images, soft_labels, epochs=30)

Epoch 1/30
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1342 - accuracy: 0.7802
Epoch 2/30
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1034 - accuracy: 0.9199
Epoch 3/30
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1009 - accuracy: 0.9339
Epoch 4/30
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1000 - accuracy: 0.9388
Epoch 5/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0995 - accuracy: 0.9409
Epoch 6/30
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0989 - accuracy: 0.9432
Epoch 7/30
 554/1875 [=======>......................] - ETA: 3s - loss: 0.0990 - accuracy: 0.9455